# Importação das Bibliotecas
- pickle: Biblioteca padrão do Python para serialização e desserialização de objetos
- pathlib: Interface orientada a objetos para trabalhar com caminhos de arquivos e diretórios
- sklearn: Biblioteca com modelos, pré-processamento, avaliação e otimazação 

In [9]:
import pickle 
import pandas as pd
import numpy as np
import ast
from pathlib import Path 
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from gensim.models import Word2Vec
import spacy
nlp = spacy.load("en_core_web_sm", disable=["ner", "parser"])

# Separação entre dados de treino e dados de teste

In [10]:
df = pd.read_csv("../../data/processed/IMDB_cleaned_SpaCy.csv")

print(df.columns)                 
print(type(df['tokens'][0])) # Problema: está em formato de string, deve se converter para lista com strings

df['tokens'] = df['tokens'].apply(ast.literal_eval)
print(type(df['tokens'][0]))

Index(['review', 'sentiment', 'clean_review', 'tokens'], dtype='object')
<class 'str'>
<class 'list'>


In [11]:
df = pd.read_csv("../../data/processed/IMDB_cleaned_SpaCy.csv")
X = df['tokens']  
y = df['sentiment']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Cria uma pasta para salvar os arquivos vetorizados
save_path = Path("../../data/vectorized")
save_path.mkdir(parents=True, exist_ok=True)

In [12]:
def save_vector(vectorization_type):
    (save_path / f"{vectorization_type}").mkdir(parents=True, exist_ok=True)
    pickle.dump((vectorization_type, X_train_vector, X_test_vector), open(save_path / f"{vectorization_type}" / f"{vectorization_type}.pkl", "wb"))
    pickle.dump(y_train, open(save_path / f"{vectorization_type}" / f"y_train_{vectorization_type}.pkl", "wb"))
    pickle.dump(y_test, open(save_path / f"{vectorization_type}" / f"y_test_{vectorization_type}.pkl", "wb"))


# Vetorização
- Bag of Words
- TF-IDF
- N-grams
- Word2vec

In [13]:
# Bag of Words: Vetorização e salvamento dos arquivos
bow_vectorizer = CountVectorizer(max_features=10000)
X_train_vector = bow_vectorizer.fit_transform(X_train)
X_test_vector = bow_vectorizer.transform(X_test)

save_vector("bow")

In [14]:
# TF-IDF: Vetorização e salvamento dos arquivos
tfidf_vectorizer = TfidfVectorizer(max_features=10000)
X_train_vector = tfidf_vectorizer.fit_transform(X_train)
X_test_vector = tfidf_vectorizer.transform(X_test)

save_vector("tfidf")

In [15]:
# N-grams (bigrama): Vetorização e salvamento dos arquivos
ngram_vectorizer = CountVectorizer(max_features=10000, ngram_range=(1, 2))
X_train_vector = ngram_vectorizer.fit_transform(X_train)
X_test_vector = ngram_vectorizer.transform(X_test)

save_vector("ngram")

In [16]:
# Treinar modelo Word2Vec: máximo de 100 dimensões, 5 palavras de contexto na esquerda e direita, 
# ignora palavras que aparecem menos de 2 vezes, usa 4 threads para acelerar o treino, usa Skip-gram
w2v_model = Word2Vec(sentences=X_train, vector_size=100, window=5, min_count=2, workers=4, sg=1)  

# Função para vetorizar cada sentença (média dos vetores das palavras): obtém um único vetor fixo para a frase
def vectorize(tokens, model):
    vectors = [model.wv[token] for token in tokens if token in model.wv]
    if vectors:
        return np.mean(vectors, axis=0) 
    else:
        return np.zeros(model.vector_size)

# Aplicar a vetorização no treino e teste
X_train_word2vec = np.vstack([vectorize(tokens, w2v_model) for tokens in X_train])
X_test_word2vec = np.vstack([vectorize(tokens, w2v_model) for tokens in X_test])

save_vector("word2vec")